# Exploratory Notebook

In [204]:
import pandas as pd
import sqlite3
import re

In [205]:
budgets_df = pd.read_csv("../Data/tn.movie_budgets.csv.gz")

In [206]:
budgets_df

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [207]:
budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [208]:
budgets_df.describe()

,id
count,5782.000000
mean,50.372363
std,28.821076
min,1.000000
25%,25.000000
50%,50.000000
75%,75.000000
max,100.000000


In [209]:
tmdb_df = pd.read_csv("../Data/tmdb.movies.csv.gz")

In [210]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [211]:
tmdb_df

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [212]:
genre_mapping = {
    28: "Action", 12: "Adventure", 16: "Animation", 35: "Comedy", 80: "Crime", 
    99: "Documentary", 18: "Drama", 10751: "Family", 14: "Fantasy", 36: "History", 
    27: "Horror", 10402: "Music", 9648: "Mystery", 10749: "Romance", 878: "Science Fiction", 
    10770: "TV Movie", 53: "Thriller", 10752: "War", 37: "Western"
}

In [213]:
def convert_genre_ids_to_names(ids):
    genre_ids = re.findall(r'\d+', ids)
    return ', '.join([genre_mapping.get(int(id), "Unknown") for id in genre_ids])

tmdb_df['genre_ids'] = tmdb_df['genre_ids'].apply(convert_genre_ids_to_names)

In [214]:
tmdb_df.head(10)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"Adventure, Fantasy, Family",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"Fantasy, Adventure, Animation, Family",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"Adventure, Action, Science Fiction",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"Animation, Comedy, Family",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"Action, Science Fiction, Adventure",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
5,5,"Adventure, Fantasy, Family",32657,en,Percy Jackson & the Olympians: The Lightning T...,26.691,2010-02-11,Percy Jackson & the Olympians: The Lightning T...,6.1,4229
6,6,"Action, Adventure, Fantasy, Science Fiction",19995,en,Avatar,26.526,2009-12-18,Avatar,7.4,18676
7,7,"Animation, Family, Comedy",10193,en,Toy Story 3,24.445,2010-06-17,Toy Story 3,7.7,8340
8,8,"Animation, Family, Comedy",20352,en,Despicable Me,23.673,2010-07-09,Despicable Me,7.2,10057
9,9,"Animation, Action, Comedy, Family, Science Fic...",38055,en,Megamind,22.855,2010-11-04,Megamind,6.8,3635


In [215]:
tmdb_df['original_language'].value_counts()

en    23291
fr      507
es      455
ru      298
ja      265
      ...  
mi        1
si        1
xh        1
ps        1
cr        1
Name: original_language, Length: 76, dtype: int64

In [1]:
tmdb_df = tmdb_df.loc[tmdb_df['original_language'] == "en"]

NameError: name 'tmdb_df' is not defined

In [217]:
tmdb_df['original_title'].value_counts()

Eden                      7
Home                      6
Lucky                     5
Legend                    5
Truth or Dare             5
                         ..
Thieves                   1
The Great Gatsby          1
The Good Witch's Charm    1
The House                 1
Claycat's The Thing       1
Name: original_title, Length: 21781, dtype: int64

In [218]:
tmdb_df = tmdb_df.drop_duplicates(subset=['original_title'])

In [219]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21781 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         21781 non-null  int64  
 1   genre_ids          21781 non-null  object 
 2   id                 21781 non-null  int64  
 3   original_language  21781 non-null  object 
 4   original_title     21781 non-null  object 
 5   popularity         21781 non-null  float64
 6   release_date       21781 non-null  object 
 7   title              21781 non-null  object 
 8   vote_average       21781 non-null  float64
 9   vote_count         21781 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 1.8+ MB


In [220]:
tmdb_df['release_date'].value_counts()

2010-01-01    249
2011-01-01    180
2014-01-01    146
2012-01-01    143
2013-01-01    137
             ... 
2015-12-28      1
1996-04-27      1
2008-01-01      1
2007-04-10      1
1979-12-14      1
Name: release_date, Length: 3364, dtype: int64

In [221]:
tmdb_df = tmdb_df[tmdb_df['vote_count'] >= 3000]

In [222]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374 entries, 0 to 23970
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         374 non-null    int64  
 1   genre_ids          374 non-null    object 
 2   id                 374 non-null    int64  
 3   original_language  374 non-null    object 
 4   original_title     374 non-null    object 
 5   popularity         374 non-null    float64
 6   release_date       374 non-null    object 
 7   title              374 non-null    object 
 8   vote_average       374 non-null    float64
 9   vote_count         374 non-null    int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 32.1+ KB


In [223]:
tmdb_df

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"Adventure, Fantasy, Family",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"Fantasy, Adventure, Animation, Family",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"Adventure, Action, Science Fiction",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"Animation, Comedy, Family",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"Action, Science Fiction, Adventure",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...,...
23872,23872,"Science Fiction, Mystery, Adventure",62,en,2001: A Space Odyssey,23.781,1968-04-10,2001: A Space Odyssey,8.0,5837
23875,23875,"Action, Science Fiction",336843,en,Maze Runner: The Death Cure,23.166,2018-01-26,Maze Runner: The Death Cure,7.0,3894
23882,23882,"Drama, Romance",337167,en,Fifty Shades Freed,22.512,2018-02-09,Fifty Shades Freed,6.1,3535
23916,23916,"Mystery, Thriller",401981,en,Red Sparrow,18.117,2018-03-02,Red Sparrow,6.5,3406


In [224]:
R = tmdb_df['vote_average']
v = tmdb_df['vote_count'].astype(float)
m = 3000.0
C = tmdb_df['vote_average'].mean()

tmdb_df['weighted_rating'] = (v/(v+m)) * R + (m/(v+m)) * C
tmdb_df.sort_values(by="weighted_rating", ascending=False)

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,weighted_rating
4,4,"Action, Science Fiction, Adventure",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,8.133304
23861,23861,"Drama, History, War",424,en,Schindler's List,25.334,1993-12-15,Schindler's List,8.5,8065,8.066345
23811,23811,"Adventure, Action, Fantasy",299536,en,Avengers: Infinity War,80.773,2018-04-27,Avengers: Infinity War,8.3,13948,8.052278
11032,11032,"Adventure, Drama, Science Fiction",157336,en,Interstellar,28.440,2014-11-05,Interstellar,8.2,18597,8.019494
2472,2472,"Family, Animation, Drama",8587,en,The Lion King,28.583,1994-06-23,The Lion King,8.2,10160,7.903769
...,...,...,...,...,...,...,...,...,...,...,...
17451,17451,"Action, Adventure, Science Fiction",47933,en,Independence Day: Resurgence,15.732,2016-06-24,Independence Day: Resurgence,5.1,3911,5.881595
14191,14191,"Drama, Romance, Thriller",216015,en,Fifty Shades of Grey,21.229,2015-02-13,Fifty Shades of Grey,5.4,6451,5.876310
7937,7937,"Science Fiction, Action, Adventure",82700,en,After Earth,14.415,2013-05-31,After Earth,5.1,4154,5.855047
2490,2490,"Adventure, Action, Thriller, Science Fiction",44912,en,Green Lantern,17.257,2011-06-17,Green Lantern,5.1,4290,5.840961


In [225]:
tmdb_df = tmdb_df.drop('id', axis=1)

In [226]:
tmdb_df = tmdb_df.drop('original_language', axis=1)

In [228]:
tmdb_df = tmdb_df.drop('original_title', axis=1)

In [229]:
tmdb_df

,Unnamed: 0,genre_ids,popularity,release_date,title,vote_average,vote_count,weighted_rating
0,0,"Adventure, Fantasy, Family",33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,7.526052
1,1,"Fantasy, Adventure, Animation, Family",28.734,2010-03-26,How to Train Your Dragon,7.7,7610,7.473950
2,2,"Adventure, Action, Science Fiction",28.515,2010-05-07,Iron Man 2,6.8,12368,6.819625
3,3,"Animation, Comedy, Family",28.005,1995-11-22,Toy Story,7.9,10174,7.672401
4,4,"Action, Science Fiction, Adventure",27.920,2010-07-16,Inception,8.3,22186,8.133304
...,...,...,...,...,...,...,...,...
23872,23872,"Science Fiction, Mystery, Adventure",23.781,1968-04-10,2001: A Space Odyssey,8.0,5837,7.626752
23875,23875,"Action, Science Fiction",23.166,2018-01-26,Maze Runner: The Death Cure,7.0,3894,6.956717
23882,23882,"Drama, Romance",22.512,2018-02-09,Fifty Shades Freed,6.1,3535,6.467499
23916,23916,"Mystery, Thriller",18.117,2018-03-02,Red Sparrow,6.5,3406,6.687575


In [230]:
tmdb_df.to_csv('tmdb_df.csv', index=False)

In [28]:
conn = sqlite3.connect('../Data/im.db')
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)
print(tables)

            name
0   movie_basics
1      directors
2      known_for
3     movie_akas
4  movie_ratings
5        persons
6     principals
7        writers
